In [31]:
import psycopg2
import glob


In [32]:
# Remplacez ces valeurs par vos informations de connexion
dbname = 'postgres'
user = 'postgres'
password = 'recommendation_films_oct_23_MLOPS'
host = 'localhost'


In [33]:
sql_file = sorted(glob.glob("./sql/*"))

In [34]:
movie_to_keep = [1,2,23]

In [35]:
def executesql(fichier_sql):
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    # Créer un objet cursor
    cur = conn.cursor()

    # Ouvrir le fichier SQL et exécuter son contenu
    with open(fichier_sql, 'r') as file:
        # Divise le script en commandes séparées par ';'
        sql_script = file.read()
        commands = sql_script.split(';')

        # Exécute chaque commande individuellement
        for command in commands:
            if command.strip() != '':  # Vérifie si la commande n'est pas vide
                    print(command)
                    cur.execute(command)
                    conn.commit()
    # Fermer la communication avec la base de données
    conn.commit()
    cur.close()
    conn.close()

In [36]:

for file in sql_file:
        print(file)
        executesql(file)


./sql/1-create_database.sql
CREATE TABLE genre (
	genreid serial4 NOT NULL,
	"name" varchar NOT NULL,
	CONSTRAINT genre_pkey PRIMARY KEY (genreid)
)


CREATE TABLE movie (
	movieid serial4 NOT NULL,
	title varchar NOT NULL,
	imdbid int4 NULL,
	tmdbid int4 NULL,
	"year" int4 NULL,
	decade int4 NULL,
	CONSTRAINT movie_pkey PRIMARY KEY (movieid)
)

CREATE TABLE imdb_data (
	movieid int4 NOT NULL,
	titre varchar(255) NULL,
	summary text NULL,
	certificat varchar(50) NULL,
	duration varchar(10) NULL,
	poster text NULL,
	directors varchar(500) NULL,
	writers varchar(500) NULL,
	stars varchar(500) NULL,
	CONSTRAINT imdb_data_pkey PRIMARY KEY (movieid),
	CONSTRAINT imdb_data_movieid_fkey FOREIGN KEY (movieid) REFERENCES movie(movieid)
)

CREATE INDEX movie_decade_idx ON movie USING btree (decade)

CREATE INDEX movie_year_idx ON movie USING btree (year)

CREATE TABLE model_prediction (
	modelid serial4 NOT NULL,
	class_name varchar NULL,
	params varchar NULL,
	mlflow_run_id varchar NULL,
	mlflo

In [37]:
import pandas as pd

In [38]:
rating = pd.read_csv("./ml-latest-small/ratings.csv")
movie = pd.read_csv("./ml-latest-small/movies.csv")
link= pd.read_csv("./ml-latest-small/links.csv")

In [39]:
import numpy as np
import re

In [40]:
movie=movie.merge(link,on="movieId")
movie=movie.replace(np.nan,"")

In [41]:
def get_year(row):
    title = row["title"]
    match = re.search(r'\((\d{4})\)', title)
    year = int(match.group(1)) if match else 0
    return year
def get_decade(row):
    year = int(row["year"])
    return (year//10)*10

movie['year'] = movie.apply(get_year,axis=1)
movie['decade'] = movie.apply(get_decade,axis=1)



In [42]:
movie

,movieId,title,genres,imdbId,tmdbId,year,decade
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,1995,1990
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995,1990
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,1995,1990
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,1995,1990
...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,5476944,432131.0,2017,2010
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,5914996,445030.0,2017,2010
9739,193585,Flint (2017),Drama,6397426,479308.0,2017,2010
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,8391976,483455.0,2018,2010


In [43]:
movie = movie[movie['decade'] == 1990 ].head(30)

In [44]:
movie

,movieId,title,genres,imdbId,tmdbId,year,decade
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,1995,1990
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995,1990
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,1995,1990
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,1995,1990
5,6,Heat (1995),Action|Crime|Thriller,113277,949.0,1995,1990
6,7,Sabrina (1995),Comedy|Romance,114319,11860.0,1995,1990
7,8,Tom and Huck (1995),Adventure|Children,112302,45325.0,1995,1990
8,9,Sudden Death (1995),Action,114576,9091.0,1995,1990
9,10,GoldenEye (1995),Action|Adventure|Thriller,113189,710.0,1995,1990


In [45]:
rating = rating[ rating.movieId.isin( movie.movieId)]

In [46]:
set(rating['movieId'])

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30}

In [47]:
import re

def insert_sql_movie(row):
    with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
        with conn.cursor() as cur:
            insert_query = """INSERT INTO movie (movieid, title, imdbid, tmdbid, year, decade)
            VALUES (%s, %s, %s, %s, %s, %s);
            """
            title = row["title"]
            match = re.search(r'\((\d{4})\)', title)
            year = int(match.group(1)) if match else None
            decade = (year // 10) * 10 if year else None
            
            # It's a good idea to check if year is None before proceeding
            if year is None:
                # Handle the case where the year couldn't be extracted
                print(f"Year could not be extracted for title: {title}")
                return

            data_to_insert = (
                int(row["movieId"]),
                row["title"],
                row["imdbId"],
                row["tmdbId"] if row["tmdbId"] else None,
                year,
                decade
            )
            cur.execute(insert_query, data_to_insert)

In [48]:
def insert_sql_rating(row):
    try:
        # Utilisation de gestionnaires de contexte pour la connexion et le curseur
        with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
            with conn.cursor() as cur:
                insert_query = """INSERT INTO rating (userid, movieid, rating, timestamp)
                                  VALUES (%s, %s, %s, %s);"""
                data_to_insert = (
                    int(row["userId"]),
                    int(row["movieId"]),  # Assurez-vous que movieId est également un entier
                    row["rating"],
                    row["timestamp"]
                )
                cur.execute(insert_query, data_to_insert)
    except Exception as e:
        # Gestion des erreurs en affichant un message d'erreur
        print(f"An error occurred: {e}")
        return 1
    return 0

In [49]:
movie.apply(insert_sql_movie,axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
dtype: object

In [50]:
rating.apply(insert_sql_rating,axis=1)

0        0
1        0
2        0
300      0
516      0
        ..
99497    0
99498    0
99534    0
99535    0
99536    0
Length: 1464, dtype: int64

In [51]:
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
            with conn.cursor() as cur:
                query = """select * from genre;"""
                cur.execute(query=query)
                rows = cur.fetchall()
                movie_genres = pd.DataFrame()
                for row in rows:
                    movie_genre = movie[ movie['genres'].str.contains(row[1], case=False, na=False)][["movieId"]]
                    movie_genre["genreId"]=[row[0]]*len(movie_genre)
                    movie_genres=pd.concat((movie_genres,movie_genre),axis=0)
                

/var/folders/y5/xr83lb_x16b7sc28gss89mrm0000gn/T/ipykernel_51904/982665775.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  movie_genre = movie[ movie['genres'].str.contains(row[1], case=False, na=False)][["movieId"]]


In [52]:
movie_genres

,movieId,genreId
5,6,2.0
8,9,2.0
9,10,2.0
14,15,2.0
19,20,2.0
...,...,...
9,10,18.0
19,20,18.0
20,21,18.0
21,22,18.0


In [53]:
def insert_sql_genre(row):
    try:
        with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
            with conn.cursor() as cur:
                insert_query = """INSERT INTO movie_genre ( movieid, genreid)
                                  VALUES (%s, %s);"""
                cur.execute(insert_query,(int(row["movieId"]),int(row['genreId'])))
    except Exception:
        print(
              (
               int(row["movieId"]),
               int(row['genreId'])
               )
              )

movie_genres.apply(insert_sql_genre,axis=1)


5     None
8     None
9     None
14    None
19    None
      ... 
9     None
19    None
20    None
21    None
22    None
Length: 75, dtype: object

In [54]:
#Refresh de la vue table_recap_view
try:
    with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
        with conn.cursor() as cur:
            refresh_query = """REFRESH MATERIALIZED VIEW table_recap_view;"""
            cur.execute(refresh_query)
except Exception as e:
    print("Error for refresh Table Recap {e}")

In [ ]:
for movieid in set(movie["movieId"]):
    try:
        with psycopg2.connect(dbname=dbname, user=user, password=password, host=host) as conn:
            with conn.cursor() as cur:
                refresh_query = f"""INSERT INTO imdb_data
                                (movieid, titre, summary, certificat, duration, poster, directors, writers, stars)
                                VALUES({movieid}, '', '', '', '', '', '', '', '');"""
                cur.execute(refresh_query)
    except Exception as e:
        print("Error for refresh Table Recap {e}")